In [ ]:
# import dependencies
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import inspect
import os
import random

### Define user parameters and definitions

In [ ]:
# years to train over
training_years = np.arange(2004,2020)

# directory containing the tiled training data
data_direc = '/direcc/jpflug/ML_layers/WUS_tiles/'

# range of dates to check for zero snow -- filtering ephemeral snowpack
# day = 0 is September 1
filt_range = [150,180]

# number of days in the data year
# keep this lower than 360
noDays = 330

# specify the degree tiles to train the model over
# this should be a CSV file of form (x_coord,y_coord) of the lower left coordinate
extents_file = data_direc+'degreeTilesWUS_all.csv'

# specify the maximum SWE limit to filter bad UCLA grid cells
lim = 3.0

# specify where to output the trained model outputs
modelOutputs_direc = data_direc+'temp/trial8/'

# specify whether or not to run and save outputs
# save_outputs = 0: numpy-load data, save_outputs = 1: process tile data
save_outputs = 0

# Create the directory if it doesn't exist
os.makedirs(modelOutputs_direc, exist_ok=True)

# list of SNOTEL stations to exclude from the taining data
station_list = '/direcc/jpflug/STN_data/SNOTEL_array.csv'

######### don't change this. Only used to save the parameters to a text file for reference
source_code = inspect.getsource(inspect.currentframe())
source_code_lines = source_code.split('\n')
start_line = source_code_lines.index('# Define the cell content') + 1
relevant_code = '\n'.join(source_code_lines[start_line:-5]) 
# Save the source code to a file
output_file_path = modelOutputs_direc + 'model_parameters.txt'
with open(output_file_path, 'w') as file:
    file.write(relevant_code)

### Script functions

In [ ]:
# mask ephemeral snow and bad data cells
def filter_cells(ds,filt_range,lim):
    # instantiate
    ds_mask = ds.copy()
    ds_mask2 = ds.copy()
    # ID cells where the first date already has snow
    condition = ds_mask[0, :] > 0.02
    ds_mask[filt_range[0],condition] = -1
    # ID cells with ephemeral snow
    ds_mask = ds_mask[filt_range[0]:filt_range[1],:]
    ds_mask[ds_mask < 0.02] = -1
    ds_mask[ds_mask > 0] = 0
    # sum flags to identify all bad cells
    ds_mask = np.nansum(ds_mask,axis=0)
    # filter unrealistically high SWE
    ds_mask2[ds_mask2 <= lim] = 0
    ds_mask2[ds_mask2 > lim] = -1
    ds_mask2 = np.nansum(ds_mask2,axis=0)
    ds_mask = ds_mask + ds_mask2
    return ds_mask

# remove cells overlapping snow point stations
def filter_stations(lonn,latt,stations_x,stations_y,differ_lim):
    for x,y in zip(stations_x,stations_y):
        differ = np.sqrt(((lonn-x)**2)+((latt-y)**2))
        latt[differ <= differ_lim] = np.nan
        lonn[differ <= differ_lim] = np.nan
    return lonn,latt

### Read in and prepare the SWE training targets

In [ ]:
# prepare the pandas array of degree tiles to read in
extents = pd.read_csv(extents_file,header=None)

# if preparing the data (not saved from a previous iteration of this script)
if save_outputs:
    # load the snow station locations
    stations = pd.read_csv(station_list)
    stations_x = stations['x'].values
    stations_y = stations['y'].values
    del stations
    
    # load the SWE data and make a mask array to use for the other datasets
    # flag to determine whether to concatenate data, or first time through the loop
    flag = 0
    # loop through the tiles
    for index, row in extents.iterrows():
        # lower-left coordinate of the tile
        ll_x = row[0]
        ll_y = row[1]
        # loop through the training years
        for yearCount,year in enumerate(training_years):
            print(year,ll_x,ll_y)

            # read in the xarray tile for the corresponding year and lat/lon tile
            ds = xr.open_dataset(data_direc+'UCLASWEtile_'+str(year)+'_'+str(ll_y)+'_'+str(ll_x)+'.nc')

            # prepare the spatial data arrays for lat/lon and mask grid cells with snow stations
            if yearCount == 0:
                latVals = ds['y'].values
                lonVals = ds['x'].values
                # determine the grid size
                differ_lim = (np.mean(np.diff(latVals))/2)**2
                differ_lim = differ_lim + (np.mean(np.diff(lonVals))/2)**2
                differ_lim = np.sqrt(differ_lim)
                lonVals,latVals = np.meshgrid(lonVals,latVals)
                # flatten the spatial component of the data
                latVals = latVals.reshape((-1))
                lonVals = lonVals.reshape((-1))
                # filter stations
                lonVals,latVals = filter_stations(lonVals,latVals,stations_x,stations_y,differ_lim)
               
            # flatten the data
            ds = ds['SWE'].values.reshape((noDays, -1)).astype('float16')

            # mask ephemeral snow and bad data cells
            ds_mask = filter_cells(ds,filt_range,lim) 
            ds_mask[np.isnan(lonVals)] = -1
            pctg = np.round(len(ds_mask[ds_mask < 0])/len(ds_mask)*100)
            print('percentage of data filtered:'+str(pctg))
            
            # create an array for the year
            yearVals = np.ones(ds.shape[1])*year
            yearVals = yearVals.astype(int)

            # concatenate the data arrays
            if flag == 0:
                dsSWE = ds
                dsMASK = ds_mask
                dsLON = lonVals
                dsLAT = latVals
                dsYR = yearVals
                flag = 1
            else:
                dsSWE = np.hstack((dsSWE,ds))
                dsMASK = np.hstack((dsMASK,ds_mask))
                dsLON = np.hstack((dsLON,lonVals))
                dsLAT = np.hstack((dsLAT,latVals))
                dsYR = np.hstack((dsYR,yearVals))

            # delete variables to save memory
            del ds,ds_mask,pctg

    # filter based on the SWE mask
    dsMASK = np.unique(np.where(dsMASK == 0))
    dsSWE = dsSWE[:,dsMASK]
    dsLON = dsLON[dsMASK]
    dsLAT = dsLAT[dsMASK]
    dsYR = dsYR[dsMASK]

    # save the data
    np.save(modelOutputs_direc+'SWEdata_masked.npy',dsSWE)
    np.save(modelOutputs_direc+'LONdata_masked.npy',dsLON)
    np.save(modelOutputs_direc+'LATdata_masked.npy',dsLAT)
    np.save(modelOutputs_direc+'maskArray.npy',dsMASK)
    np.save(modelOutputs_direc+'YEARdata_masked.npy',dsYR)
    
# if the data has already been prepared, and now just reading in
else:
    dsSWE = np.load(modelOutputs_direc+'SWEdata_masked.npy')
    dsLON = np.load(modelOutputs_direc+'LONdata_masked.npy')
    dsLAT = np.load(modelOutputs_direc+'LATdata_masked.npy')
    dsMASK = np.load(modelOutputs_direc+'maskArray.npy')
    dsYR = np.load(modelOutputs_direc+'YEARdata_masked.npy')

### Read in and prepare MODIS data

In [ ]:
# if preparing the data (not saved from a previous iteration of this script)
if save_outputs:
    # flag to determine whether to concatenate data, or first time through the loop
    flag = 0
    # loop through the tiles
    for index, row in extents.iterrows():
        ll_x = row[0]
        ll_y = row[1]
        # loop through the training years
        for yearCount,year in enumerate(training_years):
            print(year,ll_x,ll_y)

            # read in the xarray tile for the corresponding year and lat/lon tile
            ds = xr.open_dataset(data_direc+'MOD10Atile_'+str(year)+'_'+str(ll_y)+'_'+str(ll_x)+'.nc')
            ds = ds['SCA'].values.reshape((noDays, -1)).astype('float16')

            # concatenate the data arrays
            if flag == 0:
                dsSCF = ds
                flag = 1
            else:
                dsSCF = np.hstack((dsSCF,ds))

            del ds
        
    # filter based on the SWE mask
    dsSCF = dsSCF[:,dsMASK]
    np.save(modelOutputs_direc+'SCFdata_masked.npy',dsSCF)
    
# if the data has already been prepared, and now just reading in
else:
    dsSCF = np.load(modelOutputs_direc+'SCFdata_masked.npy')

### Read in and prepare the temperature data

In [ ]:
# if preparing the data (not saved from a previous iteration of this script)
if save_outputs:
    # flag to determine whether to concatenate data, or first time through the loop
    flag = 0
    # loop through the tiles
    for index, row in extents.iterrows():
        ll_x = row[0]
        ll_y = row[1]
        # loop through the training years
        for yearCount,year in enumerate(training_years):
            print(year,ll_x,ll_y)

            # read in the xarray tile for the corresponding year and lat/lon tile
            ds = xr.open_dataset(data_direc+'LISTtile_'+str(year)+'_'+str(ll_y)+'_'+str(ll_x)+'.nc')
            ds = ds['T'].values.reshape((noDays, -1)).astype('float16')

            # concatenate the data arrays
            if flag == 0:
                dsT = ds
                flag = 1
            else:
                dsT = np.hstack((dsT,ds))

            del ds
        
    # filter based on the SWE mask
    dsT = dsT[:,dsMASK]
    np.save(modelOutputs_direc+'Tdata_masked.npy',dsT)

# if the data has already been prepared, and now just reading in
else:
    dsT = np.load(modelOutputs_direc+'Tdata_masked.npy')

### Read in and prepare the precipitation climatology data

In [ ]:
# if preparing the data (not saved from a previous iteration of this script)
if save_outputs:
    # flag to determine whether to concatenate data, or first time through the loop
    flag = 0
    # loop through the tiles
    for index, row in extents.iterrows():
        ll_x = row[0]
        ll_y = row[1]
        
        # load the precipitation climatology for this tile
        ds = xr.open_dataset(data_direc+'LISPclimo_'+str(ll_y)+'_'+str(ll_x)+'.nc')
        ds = ds['P'].values.reshape((noDays, -1)).astype('float16')
        
        # loop through the training years
        for yearCount,year in enumerate(training_years):
            print(year,ll_x,ll_y)

            # concatenate the data arrays
            if flag == 0:
                dsP = ds
                flag = 1
            else:
                dsP = np.hstack((dsP,ds))
        
    # filter based on the SWE mask
    dsP = dsP[:,dsMASK]
    np.save(modelOutputs_direc+'PCLIMdata_masked.npy',dsP)
    
# if the data has already been prepared, and now just reading in
else:
    dsP = np.load(modelOutputs_direc+'PCLIMdata_masked.npy')

### Perform the snow cover post-processed layer

In [ ]:
# if preparing the data (not saved from a previous iteration of this script)
if save_outputs:
    # do the strictly-accumulating snow cover post-processing
    SCFaccum = dsSCF.copy()
    SCFaccum[np.isnan(SCFaccum)] = -9999
    
    # calculate the index of snow cover initialization
    idx_start = np.nanargmax(SCFaccum > 0.01, axis=0)
    for idxCount,idxx in enumerate(idx_start):
        SCFaccum[:idxx,idxCount] = -9999
        
    # determine max and slope to max
    idx_end = np.nanargmax(SCFaccum, axis=0)
    max_vals = np.array([SCFaccum[A,counter] for counter,A in enumerate(idx_end)])
    slope = max_vals/(idx_end-idx_start)

    # calcuate the post processed quasi snow accumulation
    for i in np.arange(slope.shape[0]):
        SCFaccum[idx_start[i]:idx_end[i],i] = (np.arange(idx_start[i],idx_end[i])*slope[i])-(idx_start[i]*slope[i])
    np.save(modelOutputs_direc+'SCFaccum_masked.npy',SCFaccum)
    
# if the data has already been prepared, and now just reading in
else:
    SCFaccum = np.load(modelOutputs_direc+'SCFaccum_masked.npy')

### Final data prep: ID nodata, normalize, and save

In [ ]:
# find the lingering no data values
print('finding nodata SWE...')
nan_locs_swe = np.unique(np.where(np.isnan(dsSWE))[1])
print('finding nodata Temp...')
nan_locs_T = np.unique(np.where(np.isnan(dsT))[1])
print('finding nodata Precip...')
nan_locs_P = np.unique(np.where(np.isnan(dsP))[1])
print('combining nodata indices...')
nan_locs = np.hstack((nan_locs_swe,nan_locs_T,nan_locs_P))
filt = np.arange(dsSWE.shape[1])
mask = np.isin(filt,nan_locs,invert=True)
filt = filt[mask]

In [ ]:
# normalize the layers and fill the nodata values for snow cover
dsSWE = dsSWE[:,filt]
dsSCF = dsSCF[:,filt]
SCFaccum = SCFaccum[:,filt]
dsT = dsT[:,filt]
dsP = dsP[:,filt]
print('filtered data complete')

print('normalizing SWE...')
dsSWE = dsSWE/lim
print(np.max(dsSWE))

print('normalizing temperature...')
min_T = np.min(dsT)
max_T = np.max(dsT)
dsT = (dsT-min_T)/(max_T-min_T)
print(min_T,max_T)

print('normalizing precipitation...')
max_P = np.max(dsP)
dsP = dsP/max_P
print('0.0',max_P)

print('setting nodata for masked layers...')
dsSCF[np.isnan(dsSCF)] = -9999
dsSCF[dsSCF < 0] = -9999
SCFaccum[np.isnan(SCFaccum)] = -9999
SCFaccum[SCFaccum < 0] = -9999

In [ ]:
# save the normalization bounds
if save_outputs:
    # order of form: SWEmax,SWEanom_min,SWEanom_max,min_T,max_T,max_P
    # norm_bounds = np.array([lim,min_anom,max_anom,min_T,max_T,max_P])
    norm_bounds = np.array([lim,0,0,min_T,max_T,max_P])
    np.save(modelOutputs_direc+'normBounds.npy',norm_bounds)
    
# if the data has already been prepared, and now just reading in
else:
    norm_bounds = np.load(modelOutputs_direc+'normBounds.npy')

In [ ]:
# create a plot function for visualization, if you want
locc = random.randint(0,SCFaccum.shape[1])

fg,ax = plt.subplots()
ax.plot(np.arange(noDays),dsSWE[:,locc],'-k')
ax.plot(np.arange(noDays),dsSWE_anom[:,locc],'--k')
ax.scatter(np.arange(noDays),dsSCF[:,locc])
ax.scatter(np.arange(noDays),SCFaccum[:,locc])
ax.plot(np.arange(noDays),dsT[:,locc])
ax.plot(np.arange(noDays),dsP[:,locc])
# strng = str(dsYR[locc])+', '+str(dsLON[locc])+', '+str(dsLAT[locc])
# ax.set_title(strng)

ax.set_ylim([0,1])